In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch # แปลง ค่าพิกเซลของภาพ จากช่วง [0, 255] (ค่าของพิกเซลในภาพทั่วไป) ให้เป็นช่วง [0, 1]
from ultralytics import YOLO
import cv2
import gc
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms # ช่วยจัดการข้อมูลภาพและโหลดโมเดลสำเร็จรูป
import torchvision.datasets as datasets
import torchvision.models as models
from sklearn.metrics import classification_report, confusion_matrix #ใช้คำนวณค่าประเมินผลโมเดล เช่น Precision, Recall, F1-score
import numpy as np # ใช้จัดการข้อมูลเชิงตัวเลข (เมทริกซ์, อาเรย์)
import seaborn as sns # 	ใช้วาดกราฟที่สวยขึ้น เช่น Heatmap ของ Confusion Matrix
import matplotlib.pyplot as plt # ใช้วาดกราฟ เช่น Confusion Matrix
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import shutil
import random # ใช้สุ่มภาพจาก dataaset
from glob import glob



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
pip install torch torchvision

In [ ]:
model = YOLO('yolov8n.pt')  # ใช้โมเดล Nano เพื่อความเร็ว


100%|██████████| 6.25M/6.25M [00:00<00:00, 71.3MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# แตกไฟล์
# นำเข้าโมดูลที่จำเป็น
import zipfile
import os

with zipfile.ZipFile('/content/drive/MyDrive/garbage.v1/Garbage.v2-garbage-detection-v2.yolov8-obb.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Trashdataset')

6075 Total Images 10 class

0.battery

1.cardboard

2.clothes

3.glass

4.metal

5.miscellaneous trash (ขยะถุงดำ)

6.organic

7.paper

8.plastic

9.shoes

In [ ]:
yaml_content = """train: /content/Trashdataset/train/images
val: /content/Trashdataset/valid/images
test: /content/Trashdataset/test/images

nc: 4
names: ["general", "organic", "recycle", "hazardous"]
"""

with open("/content/Trashdataset/data.yaml", "w") as f:
    f.write(yaml_content)

print("สร้างไฟล์ data.yaml สำเร็จ ✅")


สร้างไฟล์ data.yaml สำเร็จ ✅


In [ ]:
!cat /content/Trashdataset/data.yaml


train: /content/Trashdataset/train/images
val: /content/Trashdataset/valid/images
test: /content/Trashdataset/test/images

nc: 4
names: ["general", "organic", "recycle", "hazardous"]


{
  0: 3,  # battery -> hazardous

  1: 2,  # cardboard -> recycle

  2: 0,  # clothes -> general

  3: 2,  # glass -> recycle

  4: 2,  # metal -> recycle

  5: 0,  # miscellaneous trash -> general

  6: 1,  # organic -> organic

  7: 2,  # paper -> recycle

  8: 2,  # plastic -> recycle
  
  9: 0   # shoes -> general
}


# กำหนดการจับคู่คลาส (Class Mapping):
โค้ดนี้ทำหน้าที่ ปรับเปลี่ยนเลข ID ของคลาส (class ID) ในไฟล์ข้อมูลสำหรับการตรวจจับวัตถุ โดยมีวัตถุประสงค์เพื่อรวมคลาสย่อยหลาย ๆ คลาสให้เป็นคลาสหลักที่ใหญ่ขึ้น



In [ ]:
import os

# แบ่งประเภทใหม่ให้ตรงกับ 4 กลุ่ม
class_mapping = {
    "battery": "hazardous",  # ขยะอันตราย
    "cardboard": "recycle",
    "clothes": "general",
    "glass": "recycle",
    "metal": "recycle",
    "miscellaneous trash": "general",
    "organic": "organic",
    "paper": "recycle",
    "plastic": "recycle",
    "shoes": "general",
}

# กำหนดค่า class_id ใหม่
new_class_list = ["general", "organic", "recycle", "hazardous"]
old_classes = ["battery", "cardboard", "clothes", "glass", "metal", "miscellaneous trash", "organic", "paper", "plastic", "shoes"]

# สร้าง dictionary ที่จับคู่ old class ID กับ new class ID
new_class_dict = {old_classes.index(k): new_class_list.index(v) for k, v in class_mapping.items()}

# แสดงผลการจับคู่ (new_class_dict)
print(new_class_dict)

base_path = "/content/Trashdataset"
subfolders = ["train", "test", "valid"]

# วนลูปในแต่ละโฟลเดอร์ train, test, valid
for subfolder in subfolders:
    label_path = os.path.join(base_path, subfolder, "labels")  # กำหนด path ใหม่

    # ตรวจสอบว่าโฟลเดอร์ labels มีอยู่จริงหรือไม่
    if not os.path.exists(label_path):
        print(f"Warning: The folder {label_path} does not exist!")
        continue

    # วนลูปในไฟล์ .txt ภายในโฟลเดอร์ labels
    for filename in os.listdir(label_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(label_path, filename)

            with open(filepath, "r") as f:
                lines = f.readlines()

            new_lines = []
            updated = False  # ตัวแปรเช็คว่าไฟล์มีการอัพเดตหรือไม่
            for line in lines:
                items = line.strip().split()
                old_class_id = int(items[0])
                if old_class_id in new_class_dict:
                    new_class_id = new_class_dict[old_class_id]
                    new_lines.append(f"{new_class_id} " + " ".join(items[1:]))
                    updated = True  # ถ้ามีการเปลี่ยนแปลง class ID
                else:
                    new_lines.append(line)

            # ถ้าเกิดการอัพเดต class ID ก็แสดงข้อความใน console
            if updated:
                print(f"Updated {filename} in {subfolder}/labels")

            # เขียนข้อมูลใหม่ลงในไฟล์
            with open(filepath, "w") as f:
                f.write("\n".join(new_lines))

Streaming output truncated to the last 5000 lines.
Updated 00014367_jpg.rf.9ec323e9c42e9b7af6f3547d2c4c678b.txt in train/labels
Updated 90_rotated_plastic_113_jpg.rf.382746ef36fead718fbd3f3427b37eac.txt in train/labels
Updated battery-536-_jpg.rf.d05c3e3d5c471296c57f3487a2f8185d.txt in train/labels
Updated cardboard-1176-_jpg.rf.06108670d4bc8a84a34a85e3c9440114.txt in train/labels
Updated 00010827_jpg.rf.f09bead797f0c53a945a2b8385b799c4.txt in train/labels
Updated format_webp-25-_jpeg_jpg.rf.5dac0bdc65c329cf4ec487ec7b75bab1.txt in train/labels
Updated 00011846_jpg.rf.5e8db02a66d617e8bd56ba80da08e9d5.txt in train/labels
Updated 60_rotated_plastic_77_jpg.rf.b4c48f0c5e9beb27b186c68a3079d9ad.txt in train/labels
Updated images-17-_jpg.rf.178e769bdef0f8051b5c6ea59ec2bea5.txt in train/labels
Updated download-9-_jpg.rf.e827adc26ced79c08adb87e50d832560.txt in train/labels
Updated 00013282_jpg.rf.51c8da81cef949c204c1a69b40a1ead0.txt in train/labels
Updated b-1829-_jpg.rf.bc3c78deef9cf717c8807b5c

# ขั้นตอนการทำงานของ mapping

วนลูปเพื่อประมวลผลไฟล์:

วนลูปผ่านโฟลเดอร์ train, test, และ valid

ในแต่ละโฟลเดอร์ จะเข้าไปในโฟลเดอร์ย่อยชื่อ labels

เปิดไฟล์ .txt ทีละไฟล์ เพื่ออ่านข้อมูล

อ่านแต่ละบรรทัดและแยกข้อมูลออกมา

นำ class ID เก่า (ตัวเลขแรกของบรรทัด) ไปตรวจสอบใน new_class_dict เพื่อหา class ID ใหม่ ที่ถูกต้อง

สร้างข้อมูลบรรทัดใหม่ด้วย class ID ใหม่ ที่ได้ และเก็บไว้ในลิสต์

เขียนข้อมูลที่ถูกอัปเดตทั้งหมดกลับไปทับไฟล์เดิม

In [ ]:
import os

# กำหนดเส้นทางไปยังโฟลเดอร์ images และ labels
image_path = "/content/Trashdataset/train/images"
label_path = "/content/Trashdataset/train/labels"

# ตรวจสอบไฟล์ภาพและ label
image_files = set([f.split(".")[0] for f in os.listdir(image_path) if f.endswith(".jpg")])
label_files = set([f.split(".")[0] for f in os.listdir(label_path) if f.endswith(".txt")])

# ตรวจสอบไฟล์ที่ขาดหาย
missing_images = label_files - image_files
missing_labels = image_files - label_files

if missing_images:
    print(f"ไฟล์ภาพต่อไปนี้ขาดหาย: {missing_images}")
if missing_labels:
    print(f"ไฟล์ label ต่อไปนี้ขาดหาย: {missing_labels}")

if not missing_images and not missing_labels:
    print("ไฟล์ภาพและ label ตรงกันทั้งหมด")

ไฟล์ภาพและ label ตรงกันทั้งหมด


พบคลาสใน label ดังนี้:   

Class 0: 1917 instances


Class 1: 1266 instances


Class 2: 3042 instances


Class 3: 1137 instances


nc: 4
names:

 ["general", "organic", "recycle", "hazardous"]

In [ ]:
from ultralytics import YOLO

# โหลดโมเดล
model = YOLO('yolov8n.pt')

# ฝึกโมเดล
results = model.train(
    data='/content/Trashdataset/data.yaml',  # เส้นทางไปยังไฟล์ data.yaml
    epochs=60,
    imgsz=640,  # ตั้งค่า imgsz ให้ตรงกับขนาดภาพใน dataset (640x640)
    batch=32,
    optimizer='Adam',  # ใช้ Adam optimizer
    lr0=0.001,  # ปรับ learning rate เริ่มต้นเป็น 0.001
    patience=5,
    augment=True,  # เปิด augmentation เพื่อเพิ่มความหลากหลายของข้อมูล
    workers=4,  # ใช้ 4 workers สำหรับ data loading (ไม่ต้องเยอะเกินไป)
    device=0,  # ใช้ GPU (หากมี)
    plots=True  # สร้างกราฟการฝึก (loss, mAP)
)

Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Trashdataset/data.yaml, epochs=60, time=None, patience=5, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 15.1MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.5MB/s]


AMP: checks passed ✅


...

      44/60       5.7G     0.8071     0.7073      1.215         49        640: 100%|██████████| 130/130 [01:15<00:00,  1.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.91it/s]

                   all       1175       2032       0.84      0.835      0.847      0.655
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 39, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



44 epochs completed in 1.073 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:18<00:00,  1.04it/s]


                   all       1175       2032      0.877      0.799      0.866      0.678
               general        373        520      0.893      0.642       0.79       0.52
               organic         71        325      0.982      0.982      0.991      0.815
               recycle        727        859      0.931      0.874      0.946      0.792
             hazardous        105        328      0.703      0.699      0.738      0.585
Speed: 0.2ms preprocess, 7.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train


# * SAVE MODEL*

In [ ]:
from google.colab import drive
import shutil

# เชื่อมต่อ Google Drive
drive.mount('/content/drive')

# คัดลอกไฟล์ best.pt ไปยัง Google Drive
shutil.copy('best.pt', '/content/drive/My Drive/TrashDetection/best.pt')

# หรือคัดลอกไฟล์ best.onnx ไปยัง Google Drive
shutil.copy('runs/detect/train/weights/best.onnx', '/content/drive/My Drive/TrashDetection/best.onnx')

In [ ]:
results = model('/content/Trashdataset/test/images/image1.jpg')
results.show()

# **Deploying a trained model on an embedded device to work with a camera.**

In [ ]:
# Import necessary libraries
import cv2  # OpenCV library for video and image processing
import torch  # PyTorch, the underlying framework for the YOLO model
import argparse  # For parsing command-line arguments
from ultralytics import YOLO  # YOLO model library

def parse_args():
    """
    Parses command-line arguments for the script.
    """
    parser = argparse.ArgumentParser(description="YOLOv12 Real-time Object Detection")
    # Argument to specify the path to the YOLO model file
    parser.add_argument('--model', type=str, default='yolov8n.pt', help='Path to yolov8n model')
    # Argument to set the confidence threshold for object detection
    parser.add_argument('--conf', type=float, default=0.25, help='Confidence threshold')
    # Argument to specify the camera index (0 for default camera)
    parser.add_argument('--camera', type=int, default=0, help='Camera index (default: 0)')
    return parser.parse_args()

def main():
    """
    Main function to run the object detection program.
    """
    args = parse_args()

    # Load the YOLO model from the specified path
    print(f"Loading model {args.model}...")
    model = YOLO(args.model)

    # Open the webcam using the specified camera index
    cap = cv2.VideoCapture(args.camera)
    if not cap.isOpened():
        print(f"Error: Could not open webcam at index {args.camera}")
        return

    # Start the main loop for real-time video processing
    while True:
        # Read a single frame from the webcam
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break

        # Resize the frame to a smaller resolution for faster processing
        frame_resized = cv2.resize(frame, (320, 240))

        # Run the YOLO detection on the resized frame
        results = model(frame_resized)

        # Iterate through the detection results
        for result in results:
            # Iterate through each detected bounding box
            for box in result.boxes:
                # Get the coordinates of the bounding box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # Get the confidence score of the detection
                conf = box.conf[0].item()
                # Get the class ID of the detected object
                cls = int(box.cls[0].item())

                # Create a label string with class ID and confidence score
                label = f"Class {cls} ({conf:.2f})"

                # Draw the bounding box rectangle on the frame
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Draw the label text above the bounding box
                cv2.putText(frame_resized, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame with the detection results
        cv2.imshow("YOLO Detection", frame_resized)

        # Check for the 'q' key press to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and destroy all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

โครงสร้างของโค้ด
การนำเข้าไลบรารี (Imports):

cv2: คือไลบรารี OpenCV ใช้สำหรับจัดการกับวิดีโอและรูปภาพ เช่น การจับภาพจากกล้อง การปรับขนาดภาพ และการวาดกรอบ

torch: คือไลบรารี PyTorch ซึ่งเป็นพื้นฐานของโมเดล YOLO ที่ใช้ในการคำนวณและประมวลผลข้อมูล

argparse: ใช้สำหรับจัดการกับอาร์กิวเมนต์ที่ส่งเข้ามาทางคอมมานด์ไลน์ ทำให้สามารถกำหนดค่าต่างๆ เช่น ชื่อโมเดลหรือความแม่นยำได้ง่ายขึ้น

ultralytics: เป็นไลบรารีที่ใช้โมเดล YOLO ทำให้การเรียกใช้งานโมเดลทำได้ง่าย

ฟังก์ชัน parse_args():

ฟังก์ชันนี้จะกำหนดพารามิเตอร์ที่สามารถปรับเปลี่ยนได้เมื่อรันโค้ด เช่น:

--model: ชื่อไฟล์โมเดล YOLO (ค่าเริ่มต้นคือ yolov8n.pt)

--conf: ค่าความแม่นยำขั้นต่ำที่ยอมรับได้ (confidence threshold) เพื่อกรองผลลัพธ์ที่ไม่น่าเชื่อถือ (ค่าเริ่มต้นคือ 0.25)

--camera: หมายเลขกล้องที่ต้องการใช้งาน (ค่าเริ่มต้นคือ 0 ซึ่งมักจะเป็นกล้องหลัก)

ฟังก์ชัน main():

เป็นฟังก์ชันหลักที่รันโปรแกรม

args = parse_args(): เรียกใช้ฟังก์ชันด้านบนเพื่อรับค่าพารามิเตอร์ต่างๆ

model = YOLO(args.model): โหลดโมเดล YOLO ที่ระบุ

cap = cv2.VideoCapture(args.camera): เปิดการใช้งานกล้องเว็บแคม

while True:: สร้างลูปไม่รู้จบเพื่อประมวลผลวิดีโอทีละเฟรม

ret, frame = cap.read(): อ่านเฟรมจากกล้อง

frame_resized = cv2.resize(...): ปรับขนาดเฟรมให้เล็กลงเพื่อความเร็วในการประมวลผล

results = model(frame_resized): ส่งเฟรมที่ปรับขนาดแล้วเข้าไปในโมเดล YOLO เพื่อตรวจจับวัตถุ

for result in results:: วนลูปเพื่อดูผลลัพธ์ของแต่ละวัตถุที่ตรวจจับได้

cv2.rectangle(...) และ cv2.putText(...): วาดกรอบสี่เหลี่ยมและข้อความระบุคลาสและความแม่นยำลงบนเฟรม

cv2.imshow(...): แสดงผลเฟรมที่มีกรอบวัตถุ

if cv2.waitKey(1) & 0xFF == ord('q'):: กดปุ่ม q เพื่อออกจากโปรแกรม

cap.release() และ cv2.destroyAllWindows(): ปิดการใช้งานกล้องและหน้าต่างแสดงผลเมื่อโปรแกรมจบการทำงาน